In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2.tools.analysis as analysisTools
import timeit
from matplotlib import rc
rc('text', usetex=True)

In [ ]:
# Main parameters
numparticles = 2
boxsize = 6 #10 #8 #6 #5 
D = 1.0
Drot = 1.0
numTrajectories = 1000
lagtime = 150
rootDirectory = '../../data/dimer/first_passage_times/'

In [ ]:
# Color definitions
c1 = '#ff3300' #'xkcd:bright orange' # 'darkorange' #'C1' # '#ff3300' # for benchmark
c2 = '#0099ff' #'royalblue' #'royalblue' # 'C0' # '#0099ff' # for MSM/RD
# Alpha transparency definition
a1 = 0.2 #0.2
a2 = 0.6 #0.9 #0.7

## Load unbound to bound FPT data (bechmark and MSMRD)

In [ ]:
# Load benchmark FPT data computed by script
numTrajs = 10000
filename1 = rootDirectory + 'patchyDimerFPTs_2bound_trajs' + str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
fptsA = []
fptsB = []
with open(filename1) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        if state == 'A':
            fptsA.append(time)
        elif state == 'B':
            fptsB.append(time)
            
# Normalize weights
weightsA = np.ones_like(fptsA)/float(len(fptsA))
weightsB = np.ones_like(fptsB)/float(len(fptsB))

In [ ]:
# Load FPT data obtained with MSMRD integrator
numTrajs = 10000
filename = rootDirectory + 'MSMRDpatchyDimerFPTs_2bound_trajs' + \
str(numTrajs) + '_lagt' + str(lagtime) +  '_boxsize' + str(boxsize) + '.xyz'
MSMRDfptsA = []
MSMRDfptsB = []
with open(filename) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        if state == 'A':
            MSMRDfptsA.append(time)
        elif state == 'B':
            MSMRDfptsB.append(time)
            
# Normalize weights
weightsAMSMRD = np.ones_like(MSMRDfptsA)/float(len(MSMRDfptsA))
weightsBMSMRD = np.ones_like(MSMRDfptsB)/float(len(MSMRDfptsB))

### Calculation of FPTs and comparisons plots

In [ ]:
# Compuate mean and standard deviation of bootstrapped samples of benchmark
calculateRates = True
numBootsrapSamples = 2000
mfptA, fptAstd = analysisTools.bootstrapping(fptsA, numBootsrapSamples)
mfptB, fptBstd = analysisTools.bootstrapping(fptsB, numBootsrapSamples)
koffA = 1.0/np.array(fptsA)
koffB = 1.0/np.array(fptsB)
print("Raw MFPTs (A and B): ", np.array(fptsA).mean(), np.array(fptsB).mean() )
print("Bootstrapping mean and std (A): ", mfptA, fptAstd)
print("Bootstrapping mean and std (B): ", mfptB, fptBstd)
# Compuate mean and standard deviation of bootstrapped samples of MSMRD simulation
MSMRDmfptA, MSMRDfptAstd = analysisTools.bootstrapping(MSMRDfptsA, numBootsrapSamples)
MSMRDmfptB, MSMRDfptBstd = analysisTools.bootstrapping(MSMRDfptsB, numBootsrapSamples)
print("Raw MSMRD MFPTs (A and B): ",np.array(MSMRDfptsA).mean(), np.array(MSMRDfptsB).mean())
print("MSMRD bootstrapping mean and std (A): ", MSMRDmfptA, MSMRDfptAstd)
print("MSMRD bootstrapping mean and std (B): ", MSMRDmfptB, MSMRDfptBstd)
print("Trajectory lengths")
print("FPTs A", len(fptsA), "FPTs MSMRD-A", len(MSMRDfptsA))
print("FPTs B", len(fptsB), "FPTs MSMRD-B", len(MSMRDfptsB))
ratioFPT = float(len(fptsA))/float(len(fptsB))
ratioMSMRDFPT = float(len(MSMRDfptsA))/float(len(MSMRDfptsB))
print("Ratios:", ratioFPT, ratioMSMRDFPT )
if calculateRates:
    mkonA = 1.0/mfptA
    mkonB = 1.0/mfptB
    MSMRDmkonA = 1.0/MSMRDmfptA
    MSMRDmkonB = 1.0/MSMRDmfptB
    konAstd = fptAstd/(mfptA*mfptA)
    konBstd = fptBstd/(mfptB*mfptB)
    MSMRDkonAstd = MSMRDfptAstd/(MSMRDmfptA*MSMRDmfptA)
    MSMRDkonBstd = MSMRDfptBstd/(MSMRDmfptB*MSMRDmfptB)
    errorKonA = np.abs(mkonA - MSMRDmkonA)
    relErrorKonA = errorKonA/mkonA
    errorKonB = np.abs(mkonB - MSMRDmkonB)
    relErrorKonB = errorKonB/mkonB
    print("On rates:")
    print("Bootstrapping mean and std (A):", mkonA, konAstd)
    print("Bootstrapping mean and std (B):", mkonB, konBstd)
    print("Bootstrapping MSMRD mean and std (A):", MSMRDmkonA, MSMRDkonAstd)
    print("Bootstrapping MSMRD mean and std (B):", MSMRDmkonB, MSMRDkonBstd)
    print("Error and relative error (A):", errorKonA, relErrorKonA)
    print("Error and relative error (B):", errorKonB, relErrorKonB)

In [ ]:
# PLOT FOR PAPER #
# Configure figure
fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True, figsize=(6,5))
fig.subplots_adjust(hspace=0)
fsize = 16
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
# Set outer common axes titles
ax0 = fig.add_subplot(111, frameon=False)
ax0.set_xlabel(r'$\mathrm{time}$', fontsize=fsize + 4)
ax0.set_ylabel(r'$\mathrm{FPTs \ distribution}$', labelpad=17, fontsize=fsize + 4)
ax0.set_xticks([0])
ax0.set_yticks([0])
hbins = range(0, 500,15)

# Plot A to unbound histogram
ax1.hist(fptsA, bins = hbins, alpha=a1, color=c1, weights = weightsA );
ax1.hist(fptsA, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step', 
         facecolor = 'None', weights = weightsA, label =r'$\mathrm{Benchmark}$');
ax1.hist(MSMRDfptsA, bins = hbins, alpha=a1, color=c2, weights = weightsAMSMRD);
ax1.hist(MSMRDfptsA, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step', 
         weights = weightsAMSMRD, label =r'$\mathrm{MSM/RD}$');
ax1.legend(fontsize=fsize, title=r'$\mathrm{Unbound} \rightarrow A$', title_fontsize = fsize, fancybox=True)
ax1.text(75, 0.13, r'$\tau = {:.2f} \pm {:.2f}$'.format(mfptA, fptAstd), fontsize=fsize, color=c1)
ax1.text(85, 0.10, r'$\tau = {:.2f} \pm {:.2f}$'.format(MSMRDmfptA, MSMRDfptAstd), fontsize=fsize, color=c2)
ax1.set_yticks(np.arange(0, 0.18, step=0.1));
ax1.set_xlim([0,500])
ax1.set_ylim([0,0.18])

# Plot B to unbound histogram
ax2.hist(fptsB, bins = hbins, alpha=a1, color=c1, weights = weightsB);
ax2.hist(fptsB, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step',
         facecolor='None', weights = weightsB, label =r'$\mathrm{Benchmark}$',);
ax2.hist(MSMRDfptsB, bins = hbins, alpha=a1, color=c2, weights = weightsBMSMRD);
ax2.hist(MSMRDfptsB, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step',
         facecolor='None', weights = weightsBMSMRD, label =r'$\mathrm{MSM/RD}$');
ax2.legend(fontsize=fsize, title=r'$\mathrm{Unbound} \rightarrow B$', title_fontsize = fsize, fancybox=True)
ax2.text(75, 0.13, r'$\tau = {:.2f} \pm {:.2f}$'.format(mfptB, fptBstd), fontsize=fsize, color=c1)
ax2.text(85, 0.10, r'$\tau = {:.2f} \pm {:.2f}$'.format(MSMRDmfptB, MSMRDfptBstd), fontsize=fsize, color=c2)
ax2.set_yticks(np.arange(0, 0.18, step=0.1));
ax2.set_xlim([0,500])
ax2.set_ylim([0,0.18])

#plt.savefig('toBound_FPTs.pdf', bbox_inches='tight')

## Load bound to unbound FPT data (bechmark and MSMRD)

In [ ]:
# Load benchmark FPT data computed by script
numTrajs = 10000
filenameA = rootDirectory + 'patchyDimerFPTs_A2unbound_trajs' + str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
filenameB = rootDirectory + 'patchyDimerFPTs_B2unbound_trajs' + str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
fptsA2unbound = []
fptsB2unbound = []
with open(filenameA) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        fptsA2unbound.append(time)
with open(filenameB) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        fptsB2unbound.append(time)
        
# Normalizing weights
weightsA2 = np.ones_like(fptsA2unbound)/float(len(fptsA2unbound))
weightsB2 = np.ones_like(fptsB2unbound)/float(len(fptsB2unbound))

In [ ]:
# Load FPT data obtained with MSMRD integrator
numTrajs = 10000
filenameA = rootDirectory + 'MSMRDpatchyDimerFPTs_A2unbound_trajs' + \
str(numTrajs) + '_lagt' + str(lagtime) +  '_boxsize' + str(boxsize) + '.xyz'
filenameB = rootDirectory + 'MSMRDpatchyDimerFPTs_B2unbound_trajs' + \
str(numTrajs) + '_lagt' + str(lagtime) +  '_boxsize' + str(boxsize) + '.xyz'
MSMRDfptsA2unbound = []
MSMRDfptsB2unbound = []
with open(filenameA) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        MSMRDfptsA2unbound.append(time)
with open(filenameB) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        MSMRDfptsB2unbound.append(time)
        
# Normalizing weights
weightsAMSMRD2 = np.ones_like(MSMRDfptsA2unbound)/float(len(MSMRDfptsA2unbound))
weightsBMSMRD2 = np.ones_like(MSMRDfptsB2unbound)/float(len(MSMRDfptsB2unbound))

### Calculation of FPTs and comparisons plots

In [ ]:
# Compuate mean and standard deviation of bootstrapped samples of benchmark
numBootsrapSamples = 2000
mfptA2unbound, fptA2unboundstd = analysisTools.bootstrapping(fptsA2unbound, numBootsrapSamples)
mfptB2unbound, fptB2unboundstd = analysisTools.bootstrapping(fptsB2unbound, numBootsrapSamples)
print("Raw MFPTs (A and B): ", np.array(fptsA2unbound).mean(), np.array(fptsB2unbound).mean() )
print("Bootstrapping mean and std (A): ", mfptA2unbound, fptA2unboundstd)
print("Bootstrapping mean and std (B): ", mfptB2unbound, fptB2unboundstd)
MSMRDmfptA2unbound, MSMRDfptA2unboundstd = analysisTools.bootstrapping(MSMRDfptsA2unbound, numBootsrapSamples)
MSMRDmfptB2unbound, MSMRDfptB2unboundstd = analysisTools.bootstrapping(MSMRDfptsB2unbound, numBootsrapSamples)
print("Raw MSMRD MFPTs (A and B): ",np.array(MSMRDfptsA2unbound).mean(), np.array(MSMRDfptsB2unbound).mean())
print("MSMRD bootstrapping mean and std (A): ", MSMRDmfptA2unbound, MSMRDfptA2unboundstd)
print("MSMRD bootstrapping mean and std (B): ", MSMRDmfptB2unbound, MSMRDfptB2unboundstd)
if calculateRates:
    mkoffA = 1.0/mfptA2unbound
    mkoffB = 1.0/mfptB2unbound
    mkoffAMSMRD = 1.0/MSMRDmfptA2unbound
    mkoffBMSMRD = 1.0/MSMRDmfptB2unbound
    koffAstd = fptA2unboundstd/(mfptA2unbound*mfptA2unbound)
    koffBstd = fptB2unboundstd/(mfptB2unbound*mfptB2unbound)
    koffAstdMSMRD = MSMRDfptA2unboundstd/(MSMRDmfptA2unbound*MSMRDmfptA2unbound)
    koffBstdMSMRD = MSMRDfptB2unboundstd/(MSMRDmfptB2unbound*MSMRDmfptB2unbound)
    # Calculate averages 
    meanKoff = (mkoffA + mkoffB)/2.0
    koffstd = (koffAstd + koffBstd)/2.0
    meanKoffMSMRD = (mkoffAMSMRD + mkoffBMSMRD)/2.0
    koffstdMSMRD = (koffAstdMSMRD + koffBstdMSMRD)/2.0
    errorKoffA = np.abs(mkoffA - mkoffAMSMRD)
    relErrorKoffA = errorKoffA/mkoffA
    errorKoffB = np.abs(mkoffB - mkoffBMSMRD)
    relErrorKoffB = errorKoffB/mkoffB
    print("Off rates:")
    print("Bootstrapping mean and std (A):", mkoffA, koffAstd)
    print("Bootstrapping mean and std (B):", mkoffB, koffBstd)
    print("Bootstrapping mean and std (Average):", meanKoff, koffstd)
    print("Bootstrapping MSMRD mean and std (A):", mkoffAMSMRD, koffAstdMSMRD)
    print("Bootstrapping MSMRD mean and std (B):", mkoffBMSMRD, koffBstdMSMRD)
    print("Bootstrapping MSMRD mean and std (Average):", meanKoffMSMRD, koffstdMSMRD)
    print("Error and relative error (A):", errorKoffA, relErrorKoffA)
    print("Error and relative error (B):", errorKoffB, relErrorKoffB)

In [ ]:
# PLOT FOR PAPER #
# Configure figure
fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True, figsize=(6,5))
fig.subplots_adjust(hspace=0)
fsize = 16
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
hbins = np.arange(0, 6, 0.2)

# Set outer common axes titles
ax0 = fig.add_subplot(111, frameon=False)
ax0.set_xlabel(r'$\mathrm{time}$', fontsize=fsize + 4)
ax0.set_ylabel(r'$\mathrm{FPTs \ distribution}$', labelpad=17, fontsize=fsize + 4)
ax0.set_xticks([0])
ax0.set_yticks([0])

# Plot A to unbound histogram
ax1.hist(fptsA2unbound, bins = hbins, alpha=a1, color=c1, weights = weightsA2);
ax1.hist(fptsA2unbound, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step',
         facecolor='None', weights = weightsA2, label = r'$\mathrm{Benchmark}$');
ax1.hist(MSMRDfptsA2unbound, bins = hbins, alpha=a1, color=c2, weights = weightsAMSMRD2);
ax1.hist(MSMRDfptsA2unbound, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step',
         facecolor='None', weights = weightsAMSMRD2, label =r'$\mathrm{MSM/RD}$');
ax1.legend(fontsize=fsize, title=r'$A \rightarrow \mathrm{Unbound}$', title_fontsize = fsize, fancybox=True)
ax1.text(1.4, 0.12, r'$\tau = {:.2f} \pm {:.2f}$'.format(mfptA2unbound, fptA2unboundstd), fontsize=fsize, color=c1)
ax1.text(1.45, 0.097, r'$\tau = {:.2f} \pm {:.2f}$'.format(MSMRDmfptA2unbound, MSMRDfptA2unboundstd), 
         fontsize=fsize, color=c2)
ax1.set_xlim([0,6])
ax1.set_ylim([0,0.15])
ax1.set_yticks(np.arange(0, 0.2, step=0.1));


# Plot B to unbound histogram
ax2.hist(fptsB2unbound, bins = hbins, alpha=a1, color=c1, weights = weightsB2);
ax2.hist(fptsB2unbound, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step',
         facecolor='None', weights = weightsB2, label =r'$\mathrm{Benchmark}$');
ax2.hist(MSMRDfptsB2unbound, bins = hbins, alpha=a1, color=c2, weights = weightsBMSMRD2);
ax2.hist(MSMRDfptsB2unbound, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step',
         facecolor='None', weights = weightsBMSMRD2, label =r'$\mathrm{MSM/RD}$');
ax2.legend(fontsize=fsize, title=r'$B \rightarrow \mathrm{Unbound}$', title_fontsize = fsize, fancybox=True)
ax2.text(1.4, 0.12, r'$\tau = {:.2f} \pm {:.2f}$'.format(mfptB2unbound, fptB2unboundstd), fontsize=fsize, color=c1)
ax2.text(1.45, 0.097, r'$\tau = {:.2f} \pm {:.2f}$'.format(MSMRDmfptB2unbound, MSMRDfptB2unboundstd), 
         fontsize=fsize, color=c2)
ax2.set_xlim([0,6])
ax2.set_ylim([0,0.15])
ax2.set_yticks(np.arange(0, 0.2, step=0.1));

#plt.savefig('toUnbound_FPTs.pdf', bbox_inches='tight')

## Load bound to bound FPT data (bechmark and MSMRD)

In [ ]:
# Load benchmark FPT data computed by script
numTrajs = 10000
filenameA = rootDirectory + 'patchyDimerFPTs_A2B_trajs' + str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
filenameB = rootDirectory + 'patchyDimerFPTs_B2A_trajs' + str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
fptsA2B = []
fptsB2A = []
with open(filenameA) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        fptsA2B.append(time)
with open(filenameB) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        fptsB2A.append(time)
        
# Normalizing weights
weightsA2B = np.ones_like(fptsA2B)/float(len(fptsA2B))
weightsB2A = np.ones_like(fptsB2A)/float(len(fptsB2A))

In [ ]:
# Load FPT data obtained with MSMRD integrator
numTrajs = 10000
filenameA = rootDirectory + 'MSMRDpatchyDimerFPTs_A2B_trajs' + \
str(numTrajs) + '_lagt' + str(lagtime) +  '_boxsize' + str(boxsize) + '.xyz'
filenameB = rootDirectory + 'MSMRDpatchyDimerFPTs_B2A_trajs' + \
str(numTrajs) + '_lagt' + str(lagtime) +  '_boxsize' + str(boxsize) + '.xyz'
MSMRDfptsA2B = []
MSMRDfptsB2A = []
with open(filenameA) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        MSMRDfptsA2B.append(time)
with open(filenameB) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        MSMRDfptsB2A.append(time)
        
# Normalizing weights
weightsA2BMSMRD = np.ones_like(MSMRDfptsA2B)/float(len(MSMRDfptsA2B))
weightsB2AMSMRD = np.ones_like(MSMRDfptsB2A)/float(len(MSMRDfptsB2A))

### Calculation of FPTs and comparisons plots

In [ ]:
# Compuate mean and standard deviation of bootstrapped samples of benchmark as well as relative errors
numBootsrapSamples = 2000
mfptA2B, fptA2Bstd = analysisTools.bootstrapping(fptsA2B, numBootsrapSamples)
mfptB2A, fptB2Astd = analysisTools.bootstrapping(fptsB2A, numBootsrapSamples)
print("Bootstrapping mean and std (A2B): ", mfptA2B, fptA2Bstd)
print("Bootstrapping mean and std (B2A): ", mfptB2A, fptB2Astd)
MSMRDmfptA2B, MSMRDfptA2Bstd = analysisTools.bootstrapping(MSMRDfptsA2B, numBootsrapSamples)
MSMRDmfptB2A, MSMRDfptB2Astd = analysisTools.bootstrapping(MSMRDfptsB2A, numBootsrapSamples)
print("MSMRD bootstrapping mean and std (A): ", MSMRDmfptA2B, MSMRDfptA2Bstd)
print("MSMRD bootstrapping mean and std (B): ", MSMRDmfptB2A, MSMRDfptB2Astd)
# Calculates A2b and B2A rates
mkA2B = 1.0/mfptA2B
mkB2A = 1.0/mfptB2A
kA2Bstd = fptA2Bstd/(mfptA2B*mfptA2B)
kB2Astd = fptB2Astd/(mfptB2A*mfptB2A)
MSMRDmkA2B = 1.0/MSMRDmfptA2B
MSMRDmkB2A = 1.0/MSMRDmfptB2A
MSMRDkA2Bstd = MSMRDfptA2Bstd/(MSMRDmfptA2B*MSMRDmfptA2B)
MSMRDkB2Astd = MSMRDfptB2Astd/(MSMRDmfptB2A*MSMRDmfptB2A)
errorKA2B = np.abs(mkA2B - MSMRDmkA2B)
relErrorKA2B = errorKA2B/mkA2B
errorKB2A = np.abs(mkB2A - MSMRDmkB2A)
relErrorKB2A = errorKB2A/mkB2A
print("On rates:")
print("Bootstrapping mean and std (A2B):", mkA2B, kA2Bstd)
print("Bootstrapping mean and std (B2A):", mkB2A, kB2Astd)
print("Bootstrapping MSMRD mean and std (A2B):", MSMRDmkA2B, MSMRDkA2Bstd)
print("Bootstrapping MSMRD mean and std (B2A):", MSMRDmkB2A, MSMRDkB2Astd)
print("Error and relative error (A2B):", errorKA2B, relErrorKA2B)
print("Error and relative error (B2A):", errorKB2A, relErrorKB2A)

In [ ]:
# PLOT FOR PAPER #
# Configure figure
fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True, figsize=(6,5))
fig.subplots_adjust(hspace=0)
fsize = 16
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
hbins = np.arange(0, 500, 15)

# Set outer common axes titles
ax0 = fig.add_subplot(111, frameon=False)
ax0.set_xlabel(r'$\mathrm{time}$', fontsize=fsize + 4)
ax0.set_ylabel(r'$\mathrm{FPTs \ distribution}$', labelpad=17, fontsize=fsize + 4)
ax0.set_xticks([0])
ax0.set_yticks([0])

# Plot A to B histogram
ax1.hist(fptsA2B, bins = hbins, alpha=a1, color=c1, weights = weightsA2B);
ax1.hist(fptsA2B, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step',
         facecolor='None', weights = weightsA2B, label = r'$\mathrm{Benchmark}$');
ax1.hist(MSMRDfptsA2B, bins = hbins, alpha=a1, color=c2, weights = weightsA2BMSMRD);
ax1.hist(MSMRDfptsA2B, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step',
         facecolor='None', weights = weightsA2BMSMRD, label =r'$\mathrm{MSM/RD}$');
ax1.legend(fontsize=fsize, title=r'$A \rightarrow B$', title_fontsize = fsize, fancybox=True)
ax1.text(60, 0.18, r'$\tau = {:.2f} \pm {:.2f}$'.format(mfptA2B, fptA2Bstd), fontsize=fsize, color=c1)
ax1.text(70, 0.13, r'$\tau = {:.2f} \pm {:.2f}$'.format(MSMRDmfptA2B, MSMRDfptA2Bstd), fontsize=fsize, color=c2)
ax1.set_xlim([0,500])
ax1.set_ylim([0,0.3])
ax1.set_yticks(np.arange(0, 0.2, step=0.1));

# Plot B to A histogram
ax2.hist(fptsB2A, bins = hbins, alpha=a1, color=c1, weights = weightsB2A);
ax2.hist(fptsB2A, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step',
         facecolor='None', weights = weightsB2A, label = r'$\mathrm{Benchmark}$');
ax2.hist(MSMRDfptsB2A, bins = hbins, alpha=a1, color=c2, weights = weightsB2AMSMRD);
ax2.hist(MSMRDfptsB2A, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step',
         facecolor='None', weights = weightsB2AMSMRD, label =r'$\mathrm{MSM/RD}$');
ax2.legend(fontsize=fsize, title=r'$B \rightarrow A$', title_fontsize = fsize, fancybox=True)
ax2.text(60, 0.18, r'$\tau = {:.2f} \pm {:.2f}$'.format(mfptB2A, fptB2Astd), fontsize=fsize, color=c1)
ax2.text(70, 0.13, r'$\tau = {:.2f} \pm {:.2f}$'.format(MSMRDmfptB2A, MSMRDfptB2Astd), fontsize=fsize, color=c2)
ax2.set_xlim([0,500])
ax2.set_ylim([0,0.3])
ax2.set_yticks(np.arange(0, 0.2, step=0.1));

#plt.savefig('bound2bound_FPTs.pdf', bbox_inches='tight')

## Load all unbound to bound FPT data for $K_{on}$ vs concentration plot

In [ ]:
# Load all data
numTrajs = 5000
boxsizes = [6, 7, 8, 9, 10, 11, 12]
numBoxes = len(boxsizes)
fpt = [[] for i in range(numBoxes)]
MSMRDfpt = [[] for i in range(numBoxes)]
for i, boxsize in enumerate(boxsizes):
    filename1 = rootDirectory + '/patchyDimerFPTs_2bound_trajs' + \
        str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
    filename2 = rootDirectory + 'MSMRDpatchyDimerFPTs_2bound_trajs' + \
        str(numTrajs) + '_lagt' + str(lagtime) +  '_boxsize' + str(boxsize) + '.xyz'
    with open(filename1) as file:
        for line in file:
            state, time = line.split(" ")
            fpt[i].append(float(time))
    with open(filename2) as file:
        for line in file:
            state, time = line.split(" ")
            MSMRDfpt[i].append(float(time))

### Calculation of FPTs and comparisons plots

In [ ]:
# Calculate meand and bootstrapping error
numBootsrapSamples = 1000
numValues = 250
meanKon = np.zeros(numBoxes)
meanKonMSMRD = np.zeros(numBoxes)
stdDevKon = np.zeros(numBoxes)
stdDevKonMSMRD = np.zeros(numBoxes)
errorKon = np.zeros(numBoxes)
errorKonMSMRD = np.zeros(numBoxes)
for i, boxsize in enumerate(boxsizes):
    # Calculate bootstrapping samples of mean first passage times
    mfpt, fptStdDev = analysisTools.bootstrapping(fpt[i], numBootsrapSamples, numValues)
    MSMRDmfpt, MSMRDfptStdDev = analysisTools.bootstrapping(MSMRDfpt[i], numBootsrapSamples, numValues)
    # Calculate on rate corresponding to bootstrapping samples
    meanKon[i], stdDevKon[i] = 1.0/mfpt, fptStdDev/(mfpt*mfpt)
    meanKonMSMRD[i], stdDevKonMSMRD[i] = 1.0/MSMRDmfpt, MSMRDfptStdDev/(MSMRDmfpt*MSMRDmfpt)
    # Percentual errors
    errorKon[i] = stdDevKon[i] #1.96*stdDevKon[i]/numBootsrapSamples
    errorKonMSMRD[i] = stdDevKonMSMRD[i] #1.96*stdDevKonMSMRD[i]/numBootsrapSamples

In [ ]:
# Plot on rates vs concentration:
fig = plt.figure()
fsize = 16
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
ax = fig.add_subplot(111)
linewidth = 2
concentrations = 1.0/pow(np.array(boxsizes),3)
# Assuming moleculas have a diameter of 5nm, the actual boxsize is 5*boxisze nm,
# lets convert to Molar concentration
avogadro = 6.022E23
oneLiter2nm = 1.0E24
concentrationsMolar = concentrations*oneLiter2nm/avogadro

# Plots with shaded regions as error bars
ax.plot(concentrationsMolar, meanKon, marker='o', markersize=6, linewidth=linewidth , color=c1, 
        label=r'$\mathrm{Benchmark}$')
ax.fill_between(concentrationsMolar, meanKon-errorKon, meanKon+errorKon, alpha=0.3, edgecolor=c1, facecolor=c1)

# Traditional error bar plots
ax.errorbar(concentrationsMolar, meanKonMSMRD, yerr=errorKonMSMRD, marker='o', markersize=6, linestyle='--',
            elinewidth=2, linewidth=linewidth, capsize=5, color=c2, label=r'$\mathrm{MSM/RD}$')

ax.set_yscale('log')
#ax.set_xlabel(r'$\mathrm{Concentration}$ ($1/\mathrm{Volume}$)', fontsize = fsize+4)
ax.set_xlabel(r'$\mathrm{Concentration}$ ($M$)', fontsize = fsize+4)
ax.set_ylabel(r'$\mathrm{log}(k_\mathrm{on}$)', fontsize = fsize+6)
ax.set_yticks([0.001,.01])
#ax.set_ylim([0,0.12])
ax.legend(fontsize=fsize, title_fontsize = fsize, fancybox=True)
#plt.savefig('kon_dimer.pdf', bbox_inches='tight')

In [ ]:
# Plot equilibrium constant vs concentration
fig = plt.figure()
fsize = 16
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
ax = fig.add_subplot(111)
linewidth = 2

# Calculate Keq and errors
Keq = meanKon/meanKoff
KeqMSMRD = meanKonMSMRD/meanKoffMSMRD
errorKeq = Keq*(errorKon/meanKon + koffstd/meanKoff)
errorKeqMSMRD = KeqMSMRD*(errorKonMSMRD/meanKonMSMRD + koffstdMSMRD/meanKoffMSMRD)

# Plots with shaded regions as error bars
ax.plot(concentrationsMolar, Keq, marker='o', markersize=6, linewidth=linewidth , color=c1, 
        label=r'$\mathrm{Benchmark}$')
ax.fill_between(concentrationsMolar, Keq-errorKeq, Keq+errorKeq, alpha=0.3, edgecolor=c1, facecolor=c1)

# Traditional error bar plots
ax.errorbar(concentrationsMolar, KeqMSMRD, yerr=errorKeqMSMRD, marker='o', markersize=6, linestyle='--',
            elinewidth=2, linewidth=linewidth, capsize=5, color=c2, label=r'$\mathrm{MSM/RD}$')

ax.set_yscale('log')
ax.set_xlabel(r'$\mathrm{Concentration}$ ($M$)', fontsize = fsize+4)
ax.set_ylabel(r'$\mathrm{log}(k_\mathrm{eq}$)', fontsize = fsize+6)
ax.legend(fontsize=fsize, title_fontsize = fsize, fancybox=True)
#plt.savefig('keq_dimer.pdf', bbox_inches='tight')

In [ ]:
# Plot on rates vs concentration, zoomed
fig = plt.figure()
fsize = 16
ax = fig.add_subplot(111)
linewidth = 2

# Plots with shaded regions as error bars
ax.plot(concentrationsMolar, meanKon, marker='o', markersize=6, linewidth=linewidth , color='C1', 
        label=r'$\mathrm{Benchmark}$')
ax.fill_between(concentrationsMolar, meanKon-errorKon, meanKon+errorKon, alpha=0.3, edgecolor='C1', facecolor='C1')
#ax.plot(concentrations, meanKonMSMRD, marker='o', markersize=6, linewidth=0*linewidth, color='C0', 
#        label=r'$\mathrm{MSM/RD}$)
#ax.fill_between(concentrations, meanKonMSMRD-errorKonMSMRD, meanKonMSMRD+errorKonMSMRD, alpha=0.35, 
#                edgecolor='C0', facecolor='C0')

# Traditional error bar plots
#ax.errorbar(concentrations, meanKon, yerr=errorKon, marker='o', markersize=6, elinewidth=2, 
#            linewidth=linewidth , capsize=5, color='C1', label=r'$\mathrm{Benchmark}$')
ax.errorbar(concentrationsMolar, meanKonMSMRD, yerr=errorKonMSMRD, marker='o', markersize=6, linestyle='--',
            elinewidth=2, linewidth=linewidth, capsize=5, color='C0', label=r'$\mathrm{MSM/RD}$')
ax.set_yscale('log')
ax.set_xlim([0.0018,0.0047])
ax.set_ylim([0.003,0.018])
ax.set_xticks([], [])
#plt.savefig('kon_dimer_zoom.pdf', bbox_inches='tight')